<a href="https://colab.research.google.com/github/eccho03/datamining_project_clothes/blob/main/A_priori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# CSV 파일 읽기
data = pd.read_csv('https://raw.githubusercontent.com/eccho03/datamining_project_clothes/refs/heads/main/size_selection_info.csv')

# 데이터 확인
print(data.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   weight   age  height size
0      62  28.0  172.72   XL
1      59  36.0  167.64    L
2      61  34.0  165.10    M
3      65  27.0  175.26    L
4      62  45.0  172.72    M


In [6]:
# 키 범주화
bins_height = [0, 150, 160, 170, 180, float('inf')]
labels_height = ['height_less_150', 'height_less_160', 'height_less_170', 'height_less_180', 'height_over_180']
data['height_range'] = pd.cut(data['height'], bins=bins_height, labels=labels_height, right=False)

# 몸무게 범주화
bins_weight = [0, 40, 50, 60, 70, 80, float('inf')]
labels_weight = ['weight_less_40', 'weight_less_50', 'weight_less_60', 'weight_less_70', 'weight_less_80', 'weight_over_80']
data['weight_range'] = pd.cut(data['weight'], bins=bins_weight, labels=labels_weight, right=False)

# 사이즈 범주화
size_mapping = {
    'XXXS': 'XS', 'XXS': 'XS', 'XS': 'XS',
    'S': 'S',
    'M': 'M',
    'L': 'L',
    'XL': 'XL', 'XXL': 'XL', 'XXXL': 'XL'
}
data['size_range'] = data['size'].replace(size_mapping)
data['size_range'] = data['size'].replace(size_mapping).fillna('Unknown') # 예외 처리

# 결과 확인
print(data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


        weight   age  height size     height_range    weight_range size_range
0           62  28.0  172.72   XL  height_less_180  weight_less_70         XL
1           59  36.0  167.64    L  height_less_170  weight_less_60          L
2           61  34.0  165.10    M  height_less_170  weight_less_70          M
3           65  27.0  175.26    L  height_less_180  weight_less_70          L
4           62  45.0  172.72    M  height_less_180  weight_less_70          M
...        ...   ...     ...  ...              ...             ...        ...
119729      63  42.0  175.26    M  height_less_180  weight_less_70          M
119730      45  29.0  154.94    S  height_less_160  weight_less_50          S
119731      61  31.0  172.72    M  height_less_180  weight_less_70          M
119732      74  31.0  167.64   XL  height_less_170  weight_less_80         XL
119733      70  30.0  167.64   XL  height_less_170  weight_less_80         XL

[119734 rows x 7 columns]


In [7]:
# 범주화된 데이터를 트랜잭션으로 변환
transactions = data[['height_range', 'weight_range', 'size']].apply(lambda x: set(x), axis=1).tolist()
print(transactions[:5])  # 첫 5개의 트랜잭션 출력

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[{'height_less_180', 'XL', 'weight_less_70'}, {'height_less_170', 'L', 'weight_less_60'}, {'height_less_170', 'weight_less_70', 'M'}, {'height_less_180', 'weight_less_70', 'L'}, {'height_less_180', 'weight_less_70', 'M'}]


In [8]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 트랜잭션 데이터를 이진화
encoded_df = pd.DataFrame([{item: (item in transaction) for item in set.union(*transactions)} for transaction in transactions])

# 데이터 확인
print(encoded_df.head())


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   height_less_180  height_over_180    NaN  weight_less_70   XXXL  \
0             True            False  False            True  False   
1            False            False  False           False  False   
2            False            False  False            True  False   
3             True            False  False            True  False   
4             True            False  False            True  False   

   weight_less_60    XXL     XL  weight_less_40  height_less_170      M  \
0           False  False   True           False            False  False   
1            True  False  False           False             True  False   
2           False  False  False           False             True   True   
3           False  False  False           False            False  False   
4           False  False  False           False            False   True   

       S      L  height_less_160  weight_less_50  weight_over_80  \
0  False  False            False           False           False  